<a href="https://colab.research.google.com/github/LiveAdityaSingh/Python-projects/blob/main/Skin_Colour_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from PIL import Image
from scipy import io
from scipy import misc

In [ ]:
def get_arm_mask(image_path):
    # Load segmentations and keep only the arms
    mask = scipy.io.loadmat(image_path)
    mask = mask.resize((256,192))
    mask = np.array(((mask==14)|(mask==15)), dtype='uint8')
    mask *= 255 # Follow PIL mask conventions (0=mask, 255=keep)
    return mask

In [ ]:

def get_color_histogram(image, mask):
    im = Image.fromarray(image)
    mask = Image.fromarray(mask)
    hist = im.histogram(mask)
    # Normalize histogram
    hist = np.asarray(hist, dtype=float)
    hist /= np.linalg.norm(hist)
    return hist

In [ ]:

def get_skintone_similarity(in_pic_path, in_seg_path, out_pic_path, out_seg_path=''):
    # Load and resize images
    in_pic = Image.open(in_pic_path)
    #in_pic = misc.imread(in_pic_path)
    in_pic = in_pic.resize((256, 192))

    out_pic = Image.open(out_pic_path)
    out_pic = out_pic.resize((256, 192))

    # Load segment and return arm mask
    in_mask  = get_arm_mask(in_seg_path)
    if out_seg_path:
        out_mask = get_arm_mask(out_seg_path)
    else:
        out_mask = in_mask

    # Get color histograms of arms
    in_hist  = get_color_histogram(in_pic, in_mask)
    out_hist = get_color_histogram(out_pic, out_mask)

    # Return dot product of histograms
    return np.dot(in_hist, out_hist)

In [ ]:
get_skintone_similarity('inputs/ftest1.jpg',
                        'sample_data/example_person.mat',
                        'inputs/ftest1.jpg',
                        'sample_data/example_person.mat')

AttributeError: ignored

In [ ]:
import numpy as np
import pickle as pkl
import scipy.io

from scipy import misc
! pip install scipy==1.1.0
from matplotlib import pyplot as plt

In [ ]:
import imageio
pic = imageio.imread('inputs/ftest3.jpg')
#pic = misc.imread()

In [ ]:

plt.imshow(pic)
plt.xticks([])
plt.yticks([])
plt.show()

1) Body mask from the human parsing model


In [ ]:

img = scipy.io.loadmat('sample_data/example_person.mat')
img = img['segment']

In [ ]:

# Keep all pixels but face/hair-related items
plt.imshow(((img!=1)&(img!=5)&(img!=0)), cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
# Keep face/hair-related items and pants
mask = ((img==1)|(img==2)|(img==4)|(img==13)|(img==9))

plt.imshow(mask, cmap='gray')
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
from PIL import Image
newsize = (256, 192)
pic = pic.resize(newsize)
# Resize in case using an image of different dimension than 256 x 192
#pic = misc.imresize(pic, (256, 192))

In [ ]:

# Stack mask into a 3-channel image so that it can be applied to RGB images
masked_img = pic*np.stack([mask]*3, axis=2)